In [1]:
from z3 import *
z3.set_param("proof", True)

In [2]:
a, b = Int("a"), Int("b")
Reach = Function("Reach", IntSort(), IntSort(), BoolSort())

chcs = [
    ForAll([a, b], Implies(And(a > 0, b > 0), Reach(a, b))),
    ForAll([a, b], Implies(Reach(a, b), Reach(b, a + b))),
    ForAll([a, b], Implies(Reach(a, b), And(a > 0, b > 0)))
]

s = SolverFor("HORN")
s.add(chcs)
s.check()

sat

In [3]:
s.model()

[Reach = [else -> And(Not(Var(1) <= 0), Not(Var(0) <= 0))]]

In [4]:
a, b = Int("a"), Int("b")
a_next, b_next = Int("a'"), Int("b'")
Reach = Function("Reach", IntSort(), IntSort(), BoolSort())

chcs = [
    ForAll([a, b], Implies(And(a > 0, b > 0), Reach(a, b))),
    ForAll([a, b, a_next, b_next], 
           Implies(And(Reach(a, b), a_next == b, b_next == a),
                   Reach(a_next, b_next))),
    ForAll([a, b], Implies(Reach(a, b), And(a > 0, b > 0)))
]

s = SolverFor("HORN")
s.add(chcs)
s.check()

sat

In [5]:
s.model()

[Reach = [else -> And(Not(Var(1) <= 0), Not(Var(0) <= 0))]]

In [6]:
import sys
sys.path.append("../../hw/base")

from verification_utils import CHCs, mk_int_array

In [7]:
chcs = CHCs([
    Implies(And(a > 0, b == 0), Reach(a, b)),
    Implies(And(Reach(a, b), a_next == b + 1, b_next == a + b),
            Reach(a_next, b_next)),
    Implies(Reach(a, b), And(a > 0, b >= 0))
])

s = chcs.create_solver()
s.model() if s.check() == sat else s.proof()

[Reach = [else -> And(Not(Var(0) <= 0), Not(Var(1) <= -1))]]

In [11]:
#import importlib
#import verification_utils; importlib.reload(verification_utils)
from verification_utils import CHCs

i, n, i_next, n_next = Ints("i n i' n'")
mem = Array("mem", IntSort(), IntSort())
mem_next = Array("mem'", IntSort(), IntSort())

Inv = Function("Inv", i.sort(), n.sort(), mem.sort(), BoolSort())

chcs = CHCs([
    Implies(n > 0, Inv(0, n, K(mem.domain(), 0))),
    Implies(And(Inv(i, n, mem), i < n), Inv(i + 1, n, Store(mem, i, n))),
    Implies(And(Inv(i, n, mem), Not(i < n)), And(mem[i] == 0))
])
chcs

"n > 0 ⇒ Inv(0, n, K(Int, 0))"
"Inv(i, n, mem) ∧ i < n ⇒ Inv(i + 1, n, Store(mem, i, n))"
"Inv(i, n, mem) ∧ ¬(i < n) ⇒ mem[i] = 0"


In [13]:
sol = chcs.solve({'fp.spacer.global': True})